# Image Bind Embedding

In [2]:
!pip install -r requirements.txt --no-cache-dir

  Cloning https://github.com/facebookresearch/pytorchvideo.git (to revision 28fe037d212663c6a24f373b94cc5d478c8c1a1d) to /tmp/pip-install-yt2904n8/pytorchvideo_78eeba6a6ab24de3a1986c62e194551a
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/pytorchvideo.git /tmp/pip-install-yt2904n8/pytorchvideo_78eeba6a6ab24de3a1986c62e194551a
  Running command git rev-parse -q --verify 'sha^28fe037d212663c6a24f373b94cc5d478c8c1a1d'
  Running command git fetch -q https://github.com/facebookresearch/pytorchvideo.git 28fe037d212663c6a24f373b94cc5d478c8c1a1d
  Running command git checkout -q 28fe037d212663c6a24f373b94cc5d478c8c1a1d
  Resolved https://github.com/facebookresearch/pytorchvideo.git to commit 28fe037d212663c6a24f373b94cc5d478c8c1a1d
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.2/890.2 MB 205.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 208.9 MB/s eta 0:00:00a 0:00:01

In [3]:
from imagebind import data
import torch
from imagebind.models import imagebind_model
from imagebind.models.imagebind_model import ModalityType

/opt/conda/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.9/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [4]:
import numpy as np
from sagemaker import get_execution_role
import boto3
import pandas as pd
from io import StringIO # Python 3.
from datasets import load_dataset,Dataset,DatasetDict,concatenate_datasets

from transformers import DataCollatorWithPadding,AutoModelForSequenceClassification, Trainer, TrainingArguments,AutoTokenizer,AutoModel,AutoConfig
from transformers.modeling_outputs import TokenClassifierOutput
import torch
import torch.nn as nn
import pandas as pd
import json

In [6]:
s3 = boto3.resource('s3')
bucket_name = 'chianglab-dataderivatives'
file_path = "mimic-iv-ed-demo-2.2/text_repr.json"


content_object = s3.Object(bucket_name, file_path)
file_content = content_object.get()['Body'].read().decode('utf-8')
json_content = json.loads(file_content)
df = pd.DataFrame(json_content).T
print("length of dataframe: "+ str(len(df)))
df.head(5)

length of dataframe: 210


,arrival,eddischarge,admission,discharge,triage,medrecon,vitals,pyxis,codes
37887480,"Patient 10014729, a 21 year old white - other ...",The ED disposition was admitted at 2125-03-19 ...,The patient was admitted at 2125-03-19 16:58:00.,The patient's discharge disposition was: home ...,"At triage: temperature was 99.1, pulse was 90....",The patient was previously taking the followin...,The patient had the following vitals: At 2125-...,The patient received the following medications...,The patient received the following diagnostic ...
34176810,"Patient 10018328, a 83 year old white female, ...",The ED disposition was admitted at 2154-02-05 ...,The patient was admitted at 2154-02-05 21:58:00.,The patient's discharge disposition was: home ...,"At triage: temperature was 97.7, pulse was 74....",The patient was previously taking the followin...,The patient had the following vitals: At 2154-...,NaN,The patient received the following diagnostic ...
32103106,"Patient 10018328, a 83 year old white female, ...",The ED disposition was home at 2154-08-03 22:2...,The patient was not admitted.,The patient was not admitted.,"At triage: temperature was 96.2, pulse was 74....",The patient was previously taking the followin...,The patient had the following vitals: At 2154-...,The patient received the following medications...,The patient received the following diagnostic ...
38797992,"Patient 10020640, a 91 year old white female, ...",The ED disposition was admitted at 2153-02-13 ...,The patient was admitted at 2153-02-13 00:22:00.,The patient's discharge disposition was: skill...,"At triage: temperature was 99.2, pulse was 130...",The patient was previously taking the followin...,The patient had the following vitals: At 2153-...,The patient received the following medications...,The patient received the following diagnostic ...
33473053,"Patient 10015272, a 78 year old white female, ...",The ED disposition was admitted at 2137-06-12 ...,The patient was admitted at 2137-06-12 18:36:00.,The patient's discharge disposition was: home ...,"At triage: temperature was 97.5, pulse was 118...",The patient was previously taking the followin...,The patient had the following vitals: At 2137-...,The patient received the following medications...,The patient received the following diagnostic ...


In [7]:
df['eddischarge'] = [1 if 'admitted' in s.lower() else 0 for s in df['eddischarge']] # admitted = 1, Home = 0
df = df[[col for col in df.columns if col != 'eddischarge'] + ['eddischarge']] # rearrange column to the end
df = df.fillna("[PAD]")
df

,arrival,admission,discharge,triage,medrecon,vitals,pyxis,codes,eddischarge
37887480,"Patient 10014729, a 21 year old white - other ...",The patient was admitted at 2125-03-19 16:58:00.,The patient's discharge disposition was: home ...,"At triage: temperature was 99.1, pulse was 90....",The patient was previously taking the followin...,The patient had the following vitals: At 2125-...,The patient received the following medications...,The patient received the following diagnostic ...,1
34176810,"Patient 10018328, a 83 year old white female, ...",The patient was admitted at 2154-02-05 21:58:00.,The patient's discharge disposition was: home ...,"At triage: temperature was 97.7, pulse was 74....",The patient was previously taking the followin...,The patient had the following vitals: At 2154-...,[PAD],The patient received the following diagnostic ...,1
32103106,"Patient 10018328, a 83 year old white female, ...",The patient was not admitted.,The patient was not admitted.,"At triage: temperature was 96.2, pulse was 74....",The patient was previously taking the followin...,The patient had the following vitals: At 2154-...,The patient received the following medications...,The patient received the following diagnostic ...,0
38797992,"Patient 10020640, a 91 year old white female, ...",The patient was admitted at 2153-02-13 00:22:00.,The patient's discharge disposition was: skill...,"At triage: temperature was 99.2, pulse was 130...",The patient was previously taking the followin...,The patient had the following vitals: At 2153-...,The patient received the following medications...,The patient received the following diagnostic ...,1
33473053,"Patient 10015272, a 78 year old white female, ...",The patient was admitted at 2137-06-12 18:36:00.,The patient's discharge disposition was: home ...,"At triage: temperature was 97.5, pulse was 118...",The patient was previously taking the followin...,The patient had the following vitals: At 2137-...,The patient received the following medications...,The patient received the following diagnostic ...,1
...,...,...,...,...,...,...,...,...,...
30272878,"Patient 10038999, a 45 year old white male, ar...",The patient was admitted at 2131-05-22 21:49:00.,The patient's discharge disposition was: home ...,"At triage: temperature was not recorded, pulse...",[PAD],The patient had the following vitals: At 2131-...,The patient received the following medications...,The patient received the following diagnostic ...,1
31628990,"Patient 10009049, a 56 year old white male, ar...",The patient was admitted at 2174-05-26 08:21:00.,The patient's discharge disposition was: home ...,"At triage: temperature was 99.0, pulse was 87....",The patient was previously taking the followin...,The patient had the following vitals: At 2174-...,The patient received the following medications...,The patient received the following diagnostic ...,1
32405286,"Patient 10004457, a 65 year old white male, ar...",The patient was admitted at 2141-08-12 16:02:00.,The patient's discharge disposition was: home ...,"At triage: temperature was 97.6, pulse was 103...",The patient was previously taking the followin...,The patient had the following vitals: At 2141-...,The patient received the following medications...,The patient received the following diagnostic ...,1
34391979,"Patient 10004720, a 61 year old white male, ar...",The patient was admitted at 2186-11-12 18:01:00.,The patient's discharge disposition was: died ...,"At triage: temperature was not recorded, pulse...",The patient was previously taking the followin...,The patient had the following vitals: At 2186-...,The patient received the following medications...,The patient received the following diagnostic ...,1


In [8]:
col_names = df.columns.drop("eddischarge")
l = []
for i in col_names:
    temp = df[[i, 'eddischarge']].reset_index()
    temp = temp.sort_values(by=['index']).reset_index() # we sort the patient ID numerically before dropping it to preserve order in encoding
    temp = temp.drop(columns=["index", "level_0"])
    temp = temp.rename(columns={i: "headline", "eddischarge": "label"})
    l.append(temp)
    print("\""+i+ "\" Dataframe")
    display(temp)

"arrival" Dataframe


,headline,label
0,"Patient 10014354, a 60 year old white male, ar...",1
1,"Patient 10010867, a 28 year old white - brazil...",1
2,"Patient 10002930, a 48 year old black/african ...",1
3,"Patient 10005866, a 57 year old portuguese mal...",1
4,"Patient 10038999, a 45 year old white male, ar...",1
...,...,...
205,"Patient 10023117, a 53 year old white male, ar...",1
206,"Patient 10002930, a 48 year old black/african ...",0
207,"Patient 10020740, a 56 year old white male, ar...",0
208,"Patient 10015860, a 53 year old white male, ar...",0


"admission" Dataframe


,headline,label
0,The patient was admitted at 2147-11-14 22:12:00.,1
1,The patient was admitted at 2147-12-30 08:40:00.,1
2,The patient was admitted at 2198-04-17 19:38:00.,1
3,The patient was admitted at 2149-09-20 14:30:00.,1
4,The patient was admitted at 2131-05-22 21:49:00.,1
...,...,...
205,The patient was admitted at 2174-06-07 23:25:00.,1
206,The patient was admitted at 2201-02-12 16:58:00.,0
207,The patient was admitted at 2150-09-15 14:09:00.,0
208,The patient was admitted at 2191-01-15 01:55:00.,0


"discharge" Dataframe


...

,headline,label
0,The patient had the following vitals: At 2147-...,1
1,The patient had the following vitals: At 2147-...,1
2,The patient had the following vitals: At 2198-...,1
3,The patient had the following vitals: At 2149-...,1
4,The patient had the following vitals: At 2131-...,1
...,...,...
205,The patient had the following vitals: At 2174-...,1
206,The patient had the following vitals: At 2201-...,0
207,The patient had the following vitals: At 2150-...,0
208,The patient had the following vitals: At 2191-...,0


"pyxis" Dataframe


,headline,label
0,The patient received the following medications...,1
1,The patient received the following medications...,1
2,The patient received the following medications...,1
3,The patient received the following medications...,1
4,The patient received the following medications...,1
...,...,...
205,[PAD],1
206,The patient received the following medications...,0
207,The patient received the following medications...,0
208,The patient received the following medications...,0


"codes" Dataframe


,headline,label
0,The patient received the following diagnostic ...,1
1,The patient received the following diagnostic ...,1
2,The patient received the following diagnostic ...,1
3,The patient received the following diagnostic ...,1
4,The patient received the following diagnostic ...,1
...,...,...
205,The patient received the following diagnostic ...,1
206,The patient received the following diagnostic ...,0
207,The patient received the following diagnostic ...,0
208,The patient received the following diagnostic ...,0


In [10]:
# Indices for sub-dataframes, that will each individually be tokenized
# 0 - arrival
# 1 - admission
# 2 - discharge
# 3 - triage
# 4 - medrecon
# 5 - vitals
# 6 - pyxis
# 7 - codes
arrival_hf=l[0]['headline'].tolist()
admission_hf=l[1]['headline'].tolist()
discharge_hf=l[2]['headline'].tolist()
triage_hf=l[3]['headline'].tolist()
medrecon_hf=l[4]['headline'].tolist()
vitals_hf=l[5]['headline'].tolist()
pyxis_hf=l[6]['headline'].tolist()
codes_hf=l[7]['headline'].tolist()

# converted to list
arrival_hf[0:5]

["Patient 10014354, a 60 year old white male, arrived via ambulance at 2147-11-14 18:22:00. The patient's marital status is single. The patient's insurance is other. The patient's language is english.",
 "Patient 10010867, a 28 year old white - brazilian female, arrived via ambulance at 2147-12-30 06:45:00. The patient's marital status is single. The patient's insurance is other. The patient's language is english.",
 "Patient 10002930, a 48 year old black/african american female, arrived via walk in at 2198-04-17 11:42:00. The patient's marital status is single. The patient's insurance is medicare. The patient's language is english.",
 "Patient 10005866, a 57 year old portuguese male, arrived via walk in at 2149-09-20 05:50:00. The patient's marital status is single. The patient's insurance is medicaid. The patient's language is english.",
 "Patient 10038999, a 45 year old white male, arrived via unknown at 2131-05-22 20:33:00. The patient's marital status is single. The patient's insu

### Instantiate ImageBind Model

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Instantiate model
model = imagebind_model.imagebind_huge(pretrained=True) # model breaks when trying to load in this model. Guessing we don't have enough RAM
model.eval()
model.to(device)

# TODO: Edit ModalityType and add all our features which is a wrapper for the text

In [ ]:
# Load data
inputs = {
    ModalityType.TEXT: data.load_and_transform_text(arrival_hf, device),
    ModalityType.TEXT: data.load_and_transform_vision_data(image_paths, device),
    ModalityType.AUDIO: data.load_and_transform_audio_data(audio_paths, device),
}

with torch.no_grad():
    embeddings = model(inputs)

    
from imagebind import data
import torch
from imagebind.models import imagebind_model
from imagebind.models.imagebind_model import ModalityType

text_list=["A dog.", "A car", "A bird"]
image_paths=[".assets/dog_image.jpg", ".assets/car_image.jpg", ".assets/bird_image.jpg"]
audio_paths=[".assets/dog_audio.wav", ".assets/car_audio.wav", ".assets/bird_audio.wav"]

device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Instantiate model
model = imagebind_model.imagebind_huge(pretrained=True)
model.eval()
model.to(device)

# Load data
inputs = {
    ModalityType.TEXT: data.load_and_transform_text(text_list, device),
    ModalityType.VISION: data.load_and_transform_vision_data(image_paths, device),
    ModalityType.AUDIO: data.load_and_transform_audio_data(audio_paths, device),
}

with torch.no_grad():
    embeddings = model(inputs)

print(
    "Vision x Text: ",
    torch.softmax(embeddings[ModalityType.VISION] @ embeddings[ModalityType.TEXT].T, dim=-1),
)
print(
    "Audio x Text: ",
    torch.softmax(embeddings[ModalityType.AUDIO] @ embeddings[ModalityType.TEXT].T, dim=-1),
)
print(
    "Vision x Audio: ",
    torch.softmax(embeddings[ModalityType.VISION] @ embeddings[ModalityType.AUDIO].T, dim=-1),
)